## ResNext 29 4x64

 - Training Dataset:  RandAugment, N=1, M=20
   Validation with Unaugmented Data
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [2]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'
!pip install tensorboard

  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-9gc5arql/apex
  Running command git clone -q https://github.com/NVIDIA/apex.git /tmp/pip-install-9gc5arql/apex
  Running command git submodule update --init --recursive -q
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200716-py3-none-any.whl size=42323 sha256=dd3d2696500bcb63ab2ef8f87ff3f2380d43c6f4752c71caa57bb0d87df5a254
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2a/b5/06/dbfaa61feab8112b9ab038603ca06f375c975e3ef464e3e063
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=192505 sha256=036b323ed580443c252a8db18b1b5b82ae92a0f67f9e3b73790b189308b6ded1
  Stored in directory: /tmp/pip-ephem-wheel-cache-7fiwhmed/wheels/2a/45/23/6b4f2d6323a65ee0022d22a96d7bf580138e689f17cc48235c
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=b33d64d04a0fd125d31551ab073871b63114be5c44d62db941a6eac998f6dec5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/93

#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_1_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00 \
    scheduler.epochs 400

#### Refine the Model

In [6]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    train.base_lr 0.001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-13 12:21:03] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_t

In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00_resume400_50/test_results_0050_cifar10

[2020-07-13 21:53:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:36<00:00,  2.17it/s]
[2020-07-13 21:54:13] __main__ INFO: Elapsed 36.43
[2020-07-13 21:54:13] __main__ INFO: Loss 0.2356 Accuracy 0.9468


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00_resume400_50/test_results_0050_cifar101

[2020-07-13 21:54:37] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.11it/s]
[2020-07-13 21:54:46] __main__ INFO: Elapsed 7.59
[2020-07-13 21:54:46] __main__ INFO: Loss 0.5942 Accuracy 0.8775


In [10]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00/test_results_0400_cifar10

[2020-07-13 21:55:02] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:36<00:00,  2.16it/s]
[2020-07-13 21:55:40] __main__ INFO: Elapsed 36.60
[2020-07-13 21:55:40] __main__ INFO: Loss 0.2970 Accuracy 0.9225


In [11]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20/exp00/test_results_0400_cifar101

[2020-07-13 21:55:50] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_cm_1_1/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.10it/s]
[2020-07-13 21:55:58] __main__ INFO: Elapsed 7.62
[2020-07-13 21:55:58] __main__ INFO: Loss 0.6040 Accuracy 0.8435


#### Record the Results

In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.6746, 0.2311, 0.1517, 0.3742],
           'Accuracy': [0.8019, 0.9321, 0.9535, 0.8905],
           'Original_Accuracy': [96.4, 96.4, 96.4, 89.6],
           'Original_CI': [(96.0, 96.7), (96.0, 96.7), (96.0, 96.7), (88.2, 90.9)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])



#df.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnext_29_4x64d,cifar10,100,0.6746,0.8019,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d,cifar10,200,0.2311,0.9321,96.4,"(96.0, 96.7)"
2,resnext_29_4x64d,cifar10,300,0.1517,0.9535,96.4,"(96.0, 96.7)"
3,resnext_29_4x64d,cifar10.1,300,0.3742,0.8905,89.6,"(88.2, 90.9)"


In [12]:
import pandas as pd
model = 'resnext_29_4x64d_ra_1_20'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', ])
c = pd.Series([model, 400, 'cifar10.1', ])

e = pd.Series([model_refined, 50, 'cifar10.1', ])
f = pd.Series([model_refined, 50, 'cifar10', ])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 96.4 if row[2] == 'cifar10' else 89.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (96.0, 96.7) if row[2] == 'cifar10' else (88.2, 90.9)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnext_29_4x64d_cm_1_1,400,cifar10,0.297,0.9225,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d_cm_1_1,400,cifar10.1,0.604,0.8435,89.6,"(88.2, 90.9)"
2,resnext_29_4x64d_cm_1_1_refined400,50,cifar10.1,0.5942,0.8775,89.6,"(88.2, 90.9)"
3,resnext_29_4x64d_cm_1_1_refined400,50,cifar10,0.2356,0.9468,96.4,"(96.0, 96.7)"


In [13]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/resnext_29_4x64d_ra_1_20'
path = '/home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_1_20'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)